In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
####################################################################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 
# 8/23/23 We want to add:
#     Take into account the RELATIVE STIM LOCATION within every barrel. Does velocity depend on distance from stimulating location?
#            New column: distance (um) from stim loc to relevant barrel pair boundary (1:2)
#            New column: distance (um) from stim loc to relevant barrel pair boundary (2:3)
#
#     METHOD
#         To calculate stim location to boundary, take these files produced by MovieMaker2.ipynb:
#             electrode.dat
#             corners_layer.dat
#             corners_barrel.dat
#         Calculate: Distance to lower barrel, distance to higher barrel,
#                    Distance to lower layer, distance to higher layer
#                       (if applicable for each)
####################################################################################

In [2]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
cell_type_prefix = 'scnn1a'
csv_prefix = cell_type_prefix + '_all_slices.csv'

enable_photoZ_interaction = True
initialize_photoZ = False
snr_cutoff_signal = 5.0
pixel_to_um_conversion = 480 / 80 # um per pixel

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/" + cell_type_prefix +"_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,L4,47.7369,49.4509,49.8291,1.551910,4.440250,3.035860,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,L4,52.7236,49.9505,51.1170,6.207490,13.131100,2.274550,...,NaN,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,L4,47.5447,49.7343,49.9818,2.209280,2.558190,5.163110,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,L4,51.1559,50.3250,51.2512,4.770840,12.652100,6.629770,...,NaN,NaN,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,04_01_06,NaN,L2/3,55.0473,52.3564,51.1973,1.274260,3.233650,4.785560,...,NaN,NaN,NaN,NaN,NaN,NaN,189.026639,198.000000,206.933090,142.367160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,6/28/2023,02_04_03,Posterior,L4,96.0000,57.9622,54.0523,0.000000,0.415489,0.306664,...,NaN,NaN,NaN,NaN,NaN,NaN,45.334980,103.683397,54.966572,139.137831
74,6/28/2023,03_01_10,Posterior,L4,50.9643,96.0000,96.0000,1.093020,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,89.704987,123.158200,257.644879,39.925056
75,6/28/2023,03_02_03,Posterior,L4,59.7131,96.0000,50.4657,0.080140,0.000000,0.730507,...,NaN,NaN,NaN,NaN,NaN,NaN,134.482759,102.498405,93.155627,118.731332
76,6/28/2023,03_03_01,Posterior,L4,96.0000,50.7756,57.9677,0.000000,1.992940,0.410993,...,NaN,NaN,NaN,NaN,NaN,NaN,67.012847,143.255826,23.814860,207.587278


In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [4]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'hidden' in subdir:
        continue
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'layer4_barrels_slice' in f[-35:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
            
        # reach into analysis##_##_## subdir (created by MovieMaker2)
        analysis_subdir = subdir + '/analysis' + rec_id
        corners_barrel_file = analysis_subdir + "/corners_barrel.dat"
        corners_layer_file = analysis_subdir + "/corners_layer.dat"
        electrode_file = analysis_subdir + "/electrode.dat"
        
        # read in barrel and layer axes data
        barrel_layer_axes = []
        for la_file in [corners_barrel_file, corners_layer_file]:
            print("\t", la_file)
            with open(la_file, 'r') as f:
                lines = f.readlines() 
            corners = [int(x) for x in lines[4:]] # the last 4 lines are diode numbers of corners
            print("\t", corners)
            layer_axes = LayerAxes(corners, verbose=False)
            print("\t", layer_axes)
            laminar_axis, laminar_axis_2 = layer_axes.get_layer_axes()
            barrel_layer_axes.append(laminar_axis)
            barrel_layer_axes.append(laminar_axis_2)
        
        # open electrode as a single integer (its diode number) in variable stim_pt
        with open(electrode_file, 'r') as f:
            lines = f.readlines()
        stim_pt = int(lines[-1]) # last line is always electrode location
        aux_obj = LaminarROI([stim_pt]).get_points()
        stim_pt = aux_obj[0]  # should be a list of len 2, representing px location [x, y]
        
        # compute perpendicular distances between each of the 4 lines and the electrode point
        perp_dists = [line_obj.get_distance_to_point(stim_pt) * pixel_to_um_conversion for line_obj in barrel_layer_axes]
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(subdir + "/" + roi_file)
            time.sleep(5)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)

        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from cell_type_prefix_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        # write perpendicular distances
        new_row['Distance to Barrel A'] = perp_dists[0]
        new_row['Distance to Barrel B'] = perp_dists[1]
        new_row['Distance to Layer nA'] = perp_dists[2]
        new_row['Distance to Layer nB'] = perp_dists[3]
        
        # open these various files and store them to the pandas dataframe
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        df = df.append(new_row, ignore_index=True)


	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-07-23\selected_zda/analysis01_01_06/corners_barrel.dat
	 [1140, 4338, 1248, 4846]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-07-23\selected_zda/analysis01_01_06/corners_layer.dat
	 [1531, 1988, 3053, 3597]


scraping data from  01_01_06.zda using layer4_barrels_slice1_rois.dat on date: 9/7/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-07-23\selected_zda/analysis01_02_03/corners_barrel.dat
	 [1148, 2983, 1496, 3093]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-07-23\selected_zda/analysis01_02_03/corners_layer.dat
	 [1299, 2063, 3051, 3896]


scraping data from  01_02_03.zda using layer4_barrels_slice1_rois.dat on date: 9/7/2023
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-07-23\selected_zda/analysis02_01_05/corners_barrel.dat
	 [2338, 4201, 1162, 2216]
	 C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/09-07-23\selected_zda/analysis02_01_05/corners_layer.da

In [5]:
# write the pandas dataframe back to csv, fill in the Stim Layer metadata, and read it back in to df
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)
input("Press Enter once you have filled in the Stim Layer metadata in " + csv_filename)

Press Enter once you have filled in the Stim Layer metadata in scnn1a_all_slices-edited.csv


''

In [6]:
# load the all-data CSV dir after you've filled in the Stim location metadata
df = pd.read_csv(data_dir + csv_filename)  # from the edited csv
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,L4,47.7369,49.4509,49.8291,1.551910,4.440250,3.03586,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,L4,52.7236,49.9505,51.1170,6.207490,13.131100,2.27455,...,NaN,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,L4,47.5447,49.7343,49.9818,2.209280,2.558190,5.16311,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,L4,51.1559,50.3250,51.2512,4.770840,12.652100,6.62977,...,NaN,NaN,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,04_01_06,NaN,L2/3,55.0473,52.3564,51.1973,1.274260,3.233650,4.78556,...,NaN,NaN,NaN,NaN,NaN,NaN,189.026639,198.000000,206.933090,142.367160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,9/7/2023,02_04_03,Posterior,L2/3,47.6409,48.9078,49.2190,2.692310,2.403120,1.92779,...,NaN,NaN,NaN,NaN,NaN,NaN,64.400289,94.218365,186.287123,34.551724
84,9/7/2023,03_01_06,Anterior,L4,47.4999,48.3735,47.3385,2.259410,3.250860,2.56239,...,NaN,NaN,NaN,NaN,NaN,NaN,132.012904,31.258292,299.960273,14.797950
85,9/7/2023,03_02_03,Anterior,L4,49.3103,47.0878,41.8927,1.618110,1.184490,1.03614,...,NaN,NaN,NaN,NaN,NaN,NaN,128.598914,28.784061,160.091531,33.369509
86,9/7/2023,03_03_05,Anterior,L4,48.4769,50.3454,50.5443,0.762933,0.878104,1.19032,...,NaN,NaN,NaN,NaN,NaN,NaN,53.210871,69.000183,162.000000,32.386070


In [7]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

Barrel pairs in L2/3
	 14 no crossing out of 60 in layer L2/3
	 5 crossing out of 60 in layer L2/3
	 41 no signal out of 60 in layer L2/3
Barrel pairs in L4
	 31 no crossing out of 98 in layer L4
	 8 crossing out of 98 in layer L4
	 59 no signal out of 98 in layer L4
Barrel pairs in L5
	 6 no crossing out of 12 in layer L5
	 2 crossing out of 12 in layer L5
	 4 no signal out of 12 in layer L5


In [8]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

Latency between barrel pairs in L2/3
	 0.545 +/- 0.417 (n = 5)
	 [0.2138 0.2299 0.7214 0.2692 1.2911]
Latency between barrel pairs in L4
	 1.188 +/- 0.874 (n = 8)
	 [2.7731 0.0717 1.222  0.9262 0.957  1.3192 0.0676 2.1705]
Latency between barrel pairs in L5
	 1.043 +/- 0.473 (n = 2)
	 [0.5704 1.5165]


In [11]:
# write the pandas dataframe back to csv
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [12]:
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,L4,47.7369,49.4509,49.8291,1.551910,4.440250,3.03586,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,L4,52.7236,49.9505,51.1170,6.207490,13.131100,2.27455,...,2.7731,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,L4,47.5447,49.7343,49.9818,2.209280,2.558190,5.16311,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,L4,51.1559,50.3250,51.2512,4.770840,12.652100,6.62977,...,NaN,0.9262,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,04_01_06,NaN,L2/3,55.0473,52.3564,51.1973,1.274260,3.233650,4.78556,...,NaN,NaN,NaN,NaN,NaN,NaN,189.026639,198.000000,206.933090,142.367160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,9/7/2023,02_04_03,Posterior,L2/3,47.6409,48.9078,49.2190,2.692310,2.403120,1.92779,...,NaN,NaN,NaN,NaN,NaN,NaN,64.400289,94.218365,186.287123,34.551724
84,9/7/2023,03_01_06,Anterior,L4,47.4999,48.3735,47.3385,2.259410,3.250860,2.56239,...,NaN,NaN,NaN,NaN,NaN,NaN,132.012904,31.258292,299.960273,14.797950
85,9/7/2023,03_02_03,Anterior,L4,49.3103,47.0878,41.8927,1.618110,1.184490,1.03614,...,NaN,NaN,NaN,NaN,NaN,NaN,128.598914,28.784061,160.091531,33.369509
86,9/7/2023,03_03_05,Anterior,L4,48.4769,50.3454,50.5443,0.762933,0.878104,1.19032,...,NaN,NaN,NaN,NaN,NaN,NaN,53.210871,69.000183,162.000000,32.386070
